# Data preparation

In diesem Notebook wird versucht sämtliche Strassennamen des [Amtliches Verzeichnis der Strassen](https://www.cadastre.ch/de/services/service/registry/street.html) mit Angaben aus Wikidata abzugleichen.  
Es werden nur Daten des Kantons BS verwendet, um die Datenmenge besser im Überblick zu behalten.

---
## Laden der Daten
Analog dem Vorgehen aus [du-StrassenVZ.ipynb](https://github.com/CaptainInler/cassda-zertifikatsarbeit/blob/main/Dataunderstanding/du-StrassenVZ.ipynb)

In [1]:
import urllib.request
    
url = 'https://data.geo.admin.ch/ch.swisstopo.amtliches-strassenverzeichnis/csv/2056/ch.swisstopo.amtliches-strassenverzeichnis.zip'
filehandle, _ = urllib.request.urlretrieve(url)

In [2]:
from zipfile import ZipFile

with ZipFile(filehandle, 'r') as zip:
    zip.printdir()
    data = zip.read("pure_str.csv")

File Name                                             Modified             Size
pure_str.csv                                   2022-10-17 03:01:38     23544021
timestamp.txt                                  2022-10-17 03:27:26           10


In [3]:
from io import StringIO
import pandas as pd

daten = StringIO(str(data,'UTF-8-SIG'))

df = pd.read_csv(daten, encoding='UTF-8-SIG', sep=';')
df.head(3)

,STR_ESID,STN_LABEL,ZIP_LABEL,COM_FOSNR,COM_NAME,COM_CANTON,STR_TYPE,STR_STATUS,STR_OFFICIAL,STR_VALID,STR_MODIFIED,STR_EASTING,STR_NORTHING
0,10258316,Eggwald,6484 Wassen UR,1220,Wassen,UR,Place,real,True,False,10.09.2022,NaN,NaN
1,10023770,Wiedenweg,4203 Grellingen,2786,Grellingen,BL,Street,real,True,True,09.09.2022,2610733.0,1254311.0
2,10179192,Wuhrbärgli,4253 Liesberg,2788,Liesberg,BL,Street,real,True,True,26.08.2022,2598709.0,1249640.0


In [4]:
dfBs = df[(df.COM_CANTON == "BS") & (df.STR_TYPE != "Area")]

In [5]:
dfBs.head(3)

,STR_ESID,STN_LABEL,ZIP_LABEL,COM_FOSNR,COM_NAME,COM_CANTON,STR_TYPE,STR_STATUS,STR_OFFICIAL,STR_VALID,STR_MODIFIED,STR_EASTING,STR_NORTHING
8636,10251567,Weinlagerstrasse,4056 Basel,2701,Basel,BS,Street,real,True,True,15.08.2022,2610180.0,1269127.0
9985,10256874,Katja Wulff-Anlage,4052 Basel,2701,Basel,BS,Place,real,True,False,16.08.2022,2612763.0,1265337.0
9989,10256872,Wibrandis Rosenblatt-Weg,4052 Basel,2701,Basel,BS,Street,real,True,False,16.08.2022,2612778.0,1265286.0


---
## Namen aufbereiten
Versuchen aus den Strassennamen von `STN_LABEL` die Namen ohne Suffix (=`Stamm`) wie strasse, weg, hof, Anlage, ... zu extrahieren

In [6]:
seperators =["strasse", "-Strasse", "weg", "-Weg", "-Anlage", "anlage", "-Promenade", "rain", "gasse", "gässlein", "gässchen",
             "-Steg", "platz", "-Platz", "-Brücke", "-Passage", "graben", "-Graben", "steg", "-Park", "park", "schanze", "tunnel", "kreisel", "ring", "allee"]

In [7]:
import numpy as np

dfBsStam = dfBs.copy() #Sonst gibt es eine Warnung beim setzen der neuen Spalte
dfBsStam["Stamm"] = dfBsStam["STN_LABEL"] #Neue Spalte, deren Werte mit dem Stamm überschrieben werden sollen
dfBsStam["suffix"] = np.nan

dfBsStam.head(3)

,STR_ESID,STN_LABEL,ZIP_LABEL,COM_FOSNR,COM_NAME,COM_CANTON,STR_TYPE,STR_STATUS,STR_OFFICIAL,STR_VALID,STR_MODIFIED,STR_EASTING,STR_NORTHING,Stamm,suffix
8636,10251567,Weinlagerstrasse,4056 Basel,2701,Basel,BS,Street,real,True,True,15.08.2022,2610180.0,1269127.0,Weinlagerstrasse,NaN
9985,10256874,Katja Wulff-Anlage,4052 Basel,2701,Basel,BS,Place,real,True,False,16.08.2022,2612763.0,1265337.0,Katja Wulff-Anlage,NaN
9989,10256872,Wibrandis Rosenblatt-Weg,4052 Basel,2701,Basel,BS,Street,real,True,False,16.08.2022,2612778.0,1265286.0,Wibrandis Rosenblatt-Weg,NaN


In [8]:
for seperator in seperators:

    dfBsStam1 = dfBsStam.STN_LABEL.str.removesuffix(seperator)
    dfBsStam2 = pd.DataFrame(dfBsStam1)
    dfBsStam2.rename(columns = {"STN_LABEL":"temp"}, inplace = True)

    dfBsStam = pd.concat([dfBsStam, dfBsStam2], axis=1)

    dfBsStam['suffix'] = np.where(dfBsStam["STN_LABEL"] != dfBsStam["temp"], seperator, dfBsStam["suffix"])
    dfBsStam['Stamm'] = np.where(dfBsStam['STN_LABEL'] != dfBsStam['temp'], dfBsStam['temp'], dfBsStam['Stamm'])

    del dfBsStam['temp']

In [9]:
dfBsStam.head(3)

,STR_ESID,STN_LABEL,ZIP_LABEL,COM_FOSNR,COM_NAME,COM_CANTON,STR_TYPE,STR_STATUS,STR_OFFICIAL,STR_VALID,STR_MODIFIED,STR_EASTING,STR_NORTHING,Stamm,suffix
8636,10251567,Weinlagerstrasse,4056 Basel,2701,Basel,BS,Street,real,True,True,15.08.2022,2610180.0,1269127.0,Weinlager,strasse
9985,10256874,Katja Wulff-Anlage,4052 Basel,2701,Basel,BS,Place,real,True,False,16.08.2022,2612763.0,1265337.0,Katja Wulff,-Anlage
9989,10256872,Wibrandis Rosenblatt-Weg,4052 Basel,2701,Basel,BS,Street,real,True,False,16.08.2022,2612778.0,1265286.0,Wibrandis Rosenblatt,-Weg


---
## Abgleich mit Wikidata
Abgleich von `Stamm` in Wikidata 

In [10]:
from SPARQLWrapper import SPARQLWrapper, JSON

URL und Header für Wikidata-Query erstellen gemäss [Access best practices](https://www.wikidata.org/wiki/Wikidata:Data_access#Access_best_practices)

In [11]:
wdUrl = "https://query.wikidata.org/sparql"
user_agent = 'Streetnamequery/1.0 (https://github.com/CaptainInler/cassda-zertifikatsarbeit)'
sparql = SPARQLWrapper(wdUrl, agent=user_agent)

In [12]:
def queryWd(sparql, subject):
    #print(wdKey)
    query = """
    SELECT ?subject ?subjectLabel ?instanceLabel WHERE {
      ?subject rdfs:label "%s"@de;
               wdt:P31 ?instance.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "de" . }   
    }
    """ % (subject)
    #print(query)
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql

Statusbar aufsetzen für eine Übersicht über die Anfragen

In [13]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)


Sollte bei der Abfrage von Wikidata ein [Statuscode 429 (Too many Requests)](https://www.wikidata.org/wiki/Wikidata:Data_access#Access_best_practices) auftreten, wird versucht den Request um die vom `retry-after` header zurückgegebene Dauer zu wiederholen.  
⚠ Sollte der Request zu oft unterbrochen werden, diesen manuell abbrechen und später wiederholen.

In [14]:
from time import sleep
from urllib.error import HTTPError

i = 0
total = int(len(dfBsStam.index))
dfr = pd.DataFrame(columns=["STR_ESID", "Name", "wikiQLabel", "wikiQ", "instance"])

maxresults = 10

for x in dfBsStam.index:
    i+=1
    #print(i)
    subject = dfBsStam['Stamm'][x]
    subjectStr = dfBsStam['STN_LABEL'][x]
    print(f"Subjekt: {subject}")
    sparql = queryWd(sparql, subject)
    
    update_progress(i / total)
    try:
        results = sparql.query()
        #print(results.info())
    except HTTPError as e:
        print(f"{i} Anfragen bis jetzt")
        #Prüfen auf Statuscode 429 (Too many Requests)
        if e.status == 429:
            print(f'Statuscode 429 aufgetreten: Anfrage geht in {e.headers.get("retry-after")}sec weiter')
            sleep(int(e.headers.get("retry-after"))+2)
        else:
            raise

    result = results.convert()
    #print(result)
    results_df = pd.json_normalize(result['results']['bindings'])
    #print(results_df)
    
    STR_ESID = dfBsStam['STR_ESID'][x]

    if not results_df.empty:
        wikiQ = results_df['subject.value'][0]
        wikiQLabel = results_df['subjectLabel.value'][0]
        instance = results_df['instanceLabel.value'][0]

        #print(f"{subjectStr} | {wikiQLabel}: {wikiQ} -> {instance}")
        dfrtemp = pd.DataFrame([[STR_ESID, subjectStr, wikiQLabel, wikiQ, instance]], columns=dfr.columns)
        dfr = pd.concat([dfr, dfrtemp])

    else:
        #print(f"{subjectStr} | {subject}: Kein Eintrag in Wikidata gefunden")
        dfrtemp = pd.DataFrame([[STR_ESID, subjectStr, subject, np.nan, np.nan]], columns=dfr.columns)
        dfr = pd.concat([dfr, dfrtemp])

    #print(x)
    #if i > maxresults:
        #break
    

Progress: [####################] 100.0%


In [15]:
dfr.reset_index(drop=True, inplace=True)
dfr.head()

,STR_ESID,Name,wikiQLabel,wikiQ,instance
0,10251567,Weinlagerstrasse,Weinlager,http://www.wikidata.org/entity/Q102011549,Innerortsstraße
1,10256874,Katja Wulff-Anlage,Katja Wulff,http://www.wikidata.org/entity/Q43267718,Mensch
2,10256872,Wibrandis Rosenblatt-Weg,Wibrandis Rosenblatt,http://www.wikidata.org/entity/Q120460,Mensch
3,10256875,Gretel Bolliger-Promenade,Gretel Bolliger,http://www.wikidata.org/entity/Q40127518,Mensch
4,10255061,Backstubenweg,Backstuben,NaN,NaN


In [20]:
dfr.to_csv("out_dp-StrassenBS-Wikidata_1_alleStrassen.csv")

---
## Strassen mit Personen exportieren
Ergebnisse nach Mensch filtern und exportieren

In [21]:
dfrmen = dfr[dfr['instance'] == "Mensch"]

dfrmen.head(10)

,STR_ESID,Name,wikiQLabel,wikiQ,instance
1,10256874,Katja Wulff-Anlage,Katja Wulff,http://www.wikidata.org/entity/Q43267718,Mensch
2,10256872,Wibrandis Rosenblatt-Weg,Wibrandis Rosenblatt,http://www.wikidata.org/entity/Q120460,Mensch
3,10256875,Gretel Bolliger-Promenade,Gretel Bolliger,http://www.wikidata.org/entity/Q40127518,Mensch
7,10255060,Helli Stehle-Weg,Helli Stehle,http://www.wikidata.org/entity/Q22687790,Mensch
64,10087588,Gottfried Keller-Strasse,Gottfried Keller,http://www.wikidata.org/entity/Q122370,Mensch
67,10025102,Paracelsusstrasse,Paracelsus,http://www.wikidata.org/entity/Q83428,Mensch
86,10025055,Friedrichstrasse,Friedrich,http://www.wikidata.org/entity/Q63086,Mensch
131,10058920,Karl Barth-Platz,Karl Barth,http://www.wikidata.org/entity/Q107473,Mensch
156,10144811,Meret Oppenheim-Strasse,Meret Oppenheim,http://www.wikidata.org/entity/Q61594,Mensch
171,10063525,Gustav Wenk-Strasse,Gustav Wenk,http://www.wikidata.org/entity/Q1556519,Mensch


In [22]:
dfrmen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 1 to 1384
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STR_ESID    62 non-null     object
 1   Name        62 non-null     object
 2   wikiQLabel  62 non-null     object
 3   wikiQ       62 non-null     object
 4   instance    62 non-null     object
dtypes: object(5)
memory usage: 2.9+ KB


In [23]:
dfrmen.to_csv("out_dp-StrassenBS-Wikidata_2_StrassenNachPersonen.csv")

**💡 Erkenntnis**  
- Lichtnelkenweg, Nachtkerzenweg, Backstubenweg: Entfernt man den suffix "weg" bleibt als Stamm der Name im Plural. Dies ist für einen 1:1 abgleich des Namens nicht von Vorteil 👯‍♂️
- Weinlager (Weinlagerstrasse) wird in Wikidata nicht erkannt. 
  - Ebenso Kabel (Kabelstrasse)
  - Maispracherweg | Maispracher: Kein Eintrag in Wikidata gefunden
- Personen werden hier gut gefunden.
  - Heisst die Strasse jedoch General Guisan-Strasse und entsprich nicht Vorname - Nachname funktioniert die Auflösung in Wikidata nicht mehr
  - Uhlandstrasse | Uhland: http://www.wikidata.org/entity/Q970954 -> City in den Vereinigten Staaten 🤔. Gemäss Strassennamen-BS: Ludwig Uhland (1787–1862), deutscher Volkskundler und Dichter
- Reichensteinerstrasse | Reichensteiner: http://www.wikidata.org/entity/Q877422 -> Rebsorte. Gemäss Strassennamen-BS: Reichenstein, Burgruine (12./13. Jh.) bei Arlesheim 🏰
- Mehrdeutigkeiten
  - Spechtweg | Specht: http://www.wikidata.org/entity/Q2308143 -> Familienname 🐦
  - Jurastrasse | Jura: http://www.wikidata.org/entity/Q45805 -> Periode 🦕
